In [28]:
import cv2
from keras.models import load_model
import numpy as np
import random
import time 

# Function to record new camera frame
# Returns normalised image
def cv2_read_camera(cap):
    ret, frame = cap.read()
    resized_frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)
    image_np = np.array(resized_frame)
    normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image

    return frame, normalized_image

# Function for multiline cv2.putText
# Converts \n into newlines
def cv2_multi_putText(img, text, **kwargs):
    # Classic arguments
    position, differential, font = (0,0), (0,0), cv2.FONT_HERSHEY_SIMPLEX

    for key, value in kwargs.items():
        if key == 'position':
            position = value
        elif key == 'differential':
            differential = value
        elif key == 'font':
            font = value
    
    for i, line in enumerate(text.split('\n')):
        # Make a new line by moving the coordinates first
        x = position[0] + i*differential[0]
        y = position[1] + i*differential[1]

        # Print text
        cv2.putText(img, line, (x, y), font, 1, 2)

# Countdown function
def countdown(t):
    
    while t:
        print(str(t) + ' seconds left...')
        time.sleep(1)
        t -= 1
      
    print('Time\'s up!')

# Rock-Paper-Scissors game as a function
def play_rock_paper_scissors():

    # Assign computer and user choices
    possible_choices = ['ROCK', 'PAPER', 'SCISSORS']
    computer_choice = random.choice(possible_choices)
    user_choice = ""

    model = load_model('keras_model.h5')
    cap = cv2.VideoCapture(1)
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Set up score
    computer_score = 0
    player_score = 0

    # Set up playing variable (when False, game stops)
    playing = True

    # Set up text settings
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    centerText = (250,250)
    fontScale              = 1
    fontColor              = (255,255,255)
    thickness              = 1
    lineType               = 2

    # #Save image
    # cv2.imwrite("out.jpg", img)

    # Loop for all rounds
    while playing:
        # Set up timer for countdown for player to select action
        timer = 21

        # Loop for each round
        while True: 
            # Check if player wants to end
            if 0xFF == ord('c'):
                playing = False
                break

            # Set up webcam input
            frame, data[0] = cv2_read_camera(cap)
            prediction = model.predict(data)

            # Print prediction
            # Assign what player has chosen
            if prediction[0][1] > 0.5:
                prediction_text = "Predicting (" + str(round(prediction[0][1] * 100, 2)) + "%) ROCK"
            elif prediction[0][2] > 0.5:
                prediction_text = "Predicting (" + str(round(prediction[0][2] * 100, 2)) + "%) PAPER" 
            elif prediction[0][3] > 0.5:
                prediction_text = "Predicting (" + str(round(prediction[0][3] * 100, 2)) + "%) SCISSORS" 
            else:
                prediction_text = "Predicting NOTHING"

            # Add timer to text
            prediction_text = str(timer) + prediction_text

            # Draw on top of image
            cv2.putText(frame,prediction_text, 
                    (50, 50), 
                    font, 
                    fontScale,
                    fontColor,
                    thickness,
                    lineType)

            # Show camera
            cv2.imshow('frame', frame)

            # Countdown and print timer
            timer = timer-1

            # Either click Q for ready, or count down
            if cv2.waitKey(1) and (timer < 0 or 0xFF == ord('q')):
                # Assign what player has chosen
                if prediction[0][1] > 0.5:
                    user_choice = "ROCK"
                elif prediction[0][2] > 0.5:
                    user_choice = "PAPER"
                elif prediction[0][3] > 0.5:
                    user_choice = "SCISSORS"
                else:
                    user_choice = "NOTHING" 
                
                break

        victor_list = ['NOBODY', 'COMPUTER', 'PLAYER']
        victor = victor_list[0]

        if (computer_choice == "ROCK" and user_choice == "SCISSORS") or (computer_choice == "SCISSORS" and user_choice == "PAPER") or (computer_choice == "PAPER" and user_choice == "ROCK"):
            victor = victor_list[1]
        elif (computer_choice == "SCISSORS" and user_choice == "ROCK") or (computer_choice == "PAPER" and user_choice == "SCISSORS") or (computer_choice == "ROCK" and user_choice == "PAPER") or (user_choice == "NOTHING"):
            victor = victor_list[2]
            
        # Give victor score, and print the winner
        victory_text = 'You have chosen... ' + user_choice + '!\nThe computer has chosen...\n'
        print(victory_text)

        # Set up webcam input
        frame, data[0] = cv2_read_camera(cap)

        # Draw on top of image
        cv2_multi_putText(frame, victory_text, position=(50, 50), differential=(0, 50))

        # Show camera
        cv2.imshow('frame', frame)

        # Wait for suspense...
        cv2.waitKey(1500)

        # Show tallied up points
        frame, data[0] = cv2_read_camera(cap)

        # Tally up points
        if (victor == 'COMPUTER'):
            computer_score += 1
        elif (victor == 'PLAYER'):
            player_score += 1

        victory_text = victory_text + computer_choice + '!\n' + victor + ' wins the round!\nComputer: ' + str(computer_score) + '\nPlayer: ' + str(player_score)
        cv2_multi_putText(frame, victory_text, position=(50, 50), differential=(0, 50))
        cv2.imshow('frame', frame)
        cv2.waitKey(3500)
        
        # Check for total winner
        if (computer_score == 3):
            frame, data[0] = cv2_read_camera(cap)
            victory_text = "COMPUTER WINS THE GAME!"
            cv2_multi_putText(frame, victory_text, position=(50, 250), differential=(0, 50))
            cv2.imshow('frame', frame)
            cv2.waitKey(3500)
            break
        elif (player_score == 3):
            frame, data[0] = cv2_read_camera(cap)
            victory_text = "PLAYER WINS THE GAME!"
            cv2_multi_putText(frame, victory_text, position=(50, 250), differential=(0, 50))
            cv2.imshow('frame', frame)
            cv2.waitKey(3500)
            break

    # After the loop release the cap object
    cap.release()

    # Destroy all windows
    cv2.destroyAllWindows()

# Play the game!
play_rock_paper_scissors()


You have chosen... PAPER!
The computer has chosen...

You have chosen... SCISSORS!
The computer has chosen...

You have chosen... PAPER!
The computer has chosen...

You have chosen... PAPER!
The computer has chosen...

You have chosen... ROCK!
The computer has chosen...

You have chosen... NOTHING!
The computer has chosen...

You have chosen... NOTHING!
The computer has chosen...

